<a href="https://colab.research.google.com/github/aneeshkrishna4739/CompanyDocAnalyser_Using_Rag/blob/main/CodebaseRag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Necessary Libraries**


In [ ]:
! pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from pinecone import Pinecone

RuntimeError: Failed to import transformers.models.auto.modeling_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
No module named 'numpy.char'

**Clone a GitHub Repo locally**

In [ ]:
github_repo = "https://github.com/aneeshkrishna4739/InSightBot"
github_repo.split("/")[-1]

'InSightBot'

In [ ]:
def clone_repository(repo_url):
    """Clones a GitHub repository to a temporary directory.

    Args:
        repo_url: The URL of the GitHub repository.

    Returns:
        The path to the cloned repository.
    """

    repo_name = github_repo.split("/")[-1]
    repo_path = f"/content/{repo_name}"
    Repo.clone_from(repo_url, str(repo_path))
    return repo_path

In [ ]:
path = clone_repository(github_repo)
path

'/content/InSightBot'

In [ ]:
path

'/content/SecureAgent'

**Define which types of files to parse and which files / folders to ignore**

In [ ]:
SUPPORTED_EXTENSIONS = {'.py', '.js', '.tsx', '.jsx', '.ipynb', '.java',
                         '.cpp', '.ts', '.go', '.rs', '.vue', '.swift', '.c', '.h'}

IGNORED_DIRS = {'node_modules', 'venv', 'env', 'dist', 'build', '.git',
                '__pycache__', '.next', '.vscode', 'vendor'}

In [ ]:
def get_file_content(file_path, repo_path):
    """
    Get content of a single file.

    Args:
        file_path (str): Path to the file

    Returns:
        Optional[Dict[str, str]]: Dictionary with file name and content
    """
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        rel_path = os.path.relpath(file_path, repo_path)

        return {
            "name": rel_path,
            "content": content
        }

    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        return None



def get_main_files_content(repo_path: str):
    """
    Get content of supported code files from the local repository.

    Args:
        repo_path: Path to the local repository

    Returns:
        List of dictionaries containing file names and contents
    """

    files_content = []

    try:

        for root, _, files in os.walk(repo_path):
            if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
                continue

            for file in files:
                file_path = os.path.join(root, file)
                if os.path.splitext(file)[1] in SUPPORTED_EXTENSIONS:
                    file_content = get_file_content(file_path, repo_path)

                    if file_content:
                        files_content.append(file_content)

    except Exception as e:
        print(e)

    return files_content

In [ ]:
files_content = get_main_files_content(path)
files_content

[{'name': 'app.py',
  'content': 'from flask import Flask, render_template, request, jsonify\nfrom groq_utils import GroqClass\nfrom preprocessing import DocumentProcessor\nfrom transcriber import YouTubeTranscriber\nfrom langchain_community.embeddings import HuggingFaceEmbeddings\nimport os\nfrom dotenv import load_dotenv\napp = Flask(__name__)\n# Load environment variables from .env file\nload_dotenv()\n\nindex_name = "insightbot"\nnamespace = "transcripts"\n\n# Initialize embeddings and shared classes\nembeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")\ngroq_client = GroqClass()  # Initialize GroqClass globally to be reused\ndocument_processor = DocumentProcessor(index_name=index_name,namespace=namespace)  # Initialize DocumentProcessor globpally\n# GLOBAL\ndocument_dir_path = "resources/documents"\ntranscript_dir_path = "resources/transcripts"\n\n@app.route(\'/\')\ndef home():\n    return render_template(\'index.html\')\n\n@app.route(\'/chatbot\

In [ ]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [ ]:
text = "I like coding"

embeddings = get_huggingface_embeddings(text)

In [ ]:
# Set the PINECONE_API_KEY as an environment variable
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index("codebase-rag")

In [ ]:
vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())


<ipython-input-47-5982ffb8f713>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())


In [ ]:
# Insert the codebase embeddings into Pinecone

documents = []

for file in files_content:
    doc = Document(
        page_content=f"{file['name']}\n\n{file['content']}",
        metadata={"source": file['name']}
    )

    documents.append(doc)


vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=HuggingFaceEmbeddings(),
    index_name="codebase-rag",
    # namespace="https://github.com/CoderAgent/SecureAgent"
    namespace=github_repo
)

<ipython-input-49-eb273c5678ab>:16: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),


In [ ]:
documents

[Document(metadata={'source': 'app.py', 'text': 'app.py\n\nfrom flask import Flask, render_template, request, jsonify\nfrom groq_utils import GroqClass\nfrom preprocessing import DocumentProcessor\nfrom transcriber import YouTubeTranscriber\nfrom langchain_community.embeddings import HuggingFaceEmbeddings\nimport os\nfrom dotenv import load_dotenv\napp = Flask(__name__)\n# Load environment variables from .env file\nload_dotenv()\n\nindex_name = "insightbot"\nnamespace = "transcripts"\n\n# Initialize embeddings and shared classes\nembeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")\ngroq_client = GroqClass()  # Initialize GroqClass globally to be reused\ndocument_processor = DocumentProcessor(index_name=index_name,namespace=namespace)  # Initialize DocumentProcessor globpally\n# GLOBAL\ndocument_dir_path = "resources/documents"\ntranscript_dir_path = "resources/transcripts"\n\n@app.route(\'/\')\ndef home():\n    return render_template(\'index.html\')\

In [ ]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=userdata.get("OPENROUTER_API_KEY")
)

In [ ]:
query = "How is the flask used?"
raw_query_embedding = get_huggingface_embeddings(query)

In [ ]:
raw_query_embedding

array([ 5.71991131e-02, -3.48081291e-02, -3.27215418e-02,  5.29010817e-02,
       -3.88054959e-02,  2.21033078e-02,  1.60938818e-02, -1.00735975e-02,
        3.04608904e-02, -6.25874922e-02,  2.71743089e-02,  3.67565900e-02,
        5.69119453e-02,  5.45354113e-02,  4.02487814e-02, -4.73744869e-02,
        3.59910820e-03,  6.65521948e-03,  1.47536704e-02,  3.57538760e-02,
        1.81228686e-02,  1.24748386e-02, -2.07926128e-02,  6.99328259e-02,
       -1.78905539e-02, -1.98267922e-02, -8.77424423e-03, -4.04769043e-03,
       -4.82026935e-02, -1.55368652e-02, -6.26485124e-02, -6.66373223e-03,
        1.43068153e-02, -4.92968187e-02,  1.30648721e-06, -2.02893210e-03,
       -4.47639599e-02,  2.07317546e-02, -2.80540297e-03,  1.37846796e-02,
        4.11506603e-03,  6.87661488e-03, -2.91272085e-02, -6.68382505e-03,
        2.94112526e-02, -4.13797721e-02,  3.90248671e-02, -5.73173016e-02,
        3.29415090e-02,  1.95522374e-03, -7.05714687e-04, -2.74959207e-02,
        8.47589783e-03,  

In [ ]:
top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(),
                                   top_k=5,
                                   include_metadata=True,
                                   namespace=github_repo)

In [ ]:
top_matches

{'matches': [{'id': 'dfb6a3d0-a630-4455-8a60-0b589003e2a6',
              'metadata': {'source': 'groq_utils/__init__.py',
                           'text': 'groq_utils/__init__.py\n'
                                   '\n'
                                   'from .GroqClass import GroqClass'},
              'score': 0.188397408,
              'values': []},
             {'id': '17048fbe-be8e-4121-a2ad-ebf339c823dd',
              'metadata': {'source': 'groq_utils/__init__.py',
                           'text': 'groq_utils/__init__.py\n'
                                   '\n'
                                   'from .GroqClass import GroqClass'},
              'score': 0.188397408,
              'values': []},
             {'id': '2173527e-7217-49cd-b7c5-776fa1049735',
              'metadata': {'source': 'app.py',
                           'text': 'app.py\n'
                                   '\n'
                                   'from flask import Flask, render_template, '
   

In [ ]:
context = [item['metadata']['text'] for item in top_matches['matches']]

In [ ]:
context

['src/context/language/javascript-parser.ts\n\nimport { AbstractParser, EnclosingContext } from "../../constants";\nimport * as parser from "@babel/parser";\nimport traverse, { NodePath, Node } from "@babel/traverse";\n\nconst processNode = (\n  path: NodePath<Node>,\n  lineStart: number,\n  lineEnd: number,\n  largestSize: number,\n  largestEnclosingContext: Node | null\n) => {\n  const { start, end } = path.node.loc;\n  if (start.line <= lineStart && lineEnd <= end.line) {\n    const size = end.line - start.line;\n    if (size > largestSize) {\n      largestSize = size;\n      largestEnclosingContext = path.node;\n    }\n  }\n  return { largestSize, largestEnclosingContext };\n};\n\nexport class JavascriptParser implements AbstractParser {\n  findEnclosingContext(\n    file: string,\n    lineStart: number,\n    lineEnd: number\n  ): EnclosingContext {\n    const ast = parser.parse(file, {\n      sourceType: "module",\n      plugins: ["jsx", "typescript"], // To allow JSX and TypeScri

In [ ]:
augmented_query = "\n" + "\n\n-------\n\n".join(context) + "\n-------\n\n\n\n\nMY QUESTION:\n" + query


In [ ]:
print(augmented_query)



src/context/language/javascript-parser.ts

import { AbstractParser, EnclosingContext } from "../../constants";
import * as parser from "@babel/parser";
import traverse, { NodePath, Node } from "@babel/traverse";

const processNode = (
  path: NodePath<Node>,
  lineStart: number,
  lineEnd: number,
  largestSize: number,
  largestEnclosingContext: Node | null
) => {
  const { start, end } = path.node.loc;
  if (start.line <= lineStart && lineEnd <= end.line) {
    const size = end.line - start.line;
    if (size > largestSize) {
      largestSize = size;
      largestEnclosingContext = path.node;
    }
  }
  return { largestSize, largestEnclosingContext };
};

export class JavascriptParser implements AbstractParser {
  findEnclosingContext(
    file: string,
    lineStart: number,
    lineEnd: number
  ): EnclosingContext {
    const ast = parser.parse(file, {
      sourceType: "module",
      plugins: ["jsx", "typescript"], // To allow JSX and TypeScript
    });
    let largestEnclosi

In [ ]:

system_prompt = """You are a Senior Software Engineer, who is an expert in TypeScript.

Answer the question I have about the codebase based on the context provided. Always consider all of the context provided
to answer my question.
"""

llm_response = client.chat.completions.create(
    model="deepseek/deepseek-r1:free",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]
)

response = llm_response.choices[0].message.content

In [ ]:
print(response)

The Flask framework is used in this codebase to create a web application with the following key functionalities:

1. **Basic Setup**:
- Initialized with `Flask(__name__)`
- Uses environment variables with `python-dotenv`
- Runs with `app.run()` using configurable host/port

2. **Routes**:
- `/`: Serves home page template (index.html)
- `/chatbot`: Serves chatbot interface template (chatbot.html)
- `/submit_media`: POST endpoint for processing media/documents
- `/ask_question`: POST endpoint for handling user queries

3. **Key Flask Features Used**:
- `request` object: Handling form data and file uploads
- `jsonify`: Returning JSON responses for API endpoints
- `render_template`: Serving HTML templates
- Route decorators (`@app.route`) for URL routing
- Form data handling with `request.form.getlist()`
- File upload handling with `request.files.getlist()`

4. **Error Handling**:
- Try/except blocks in routes
- HTTP status codes (400, 500)
- JSON error responses with error details

5. **I

In [ ]:
def perform_rag(query, model="deepseek/deepseek-r1:free"):
    raw_query_embedding = get_huggingface_embeddings(query)

    top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=5, include_metadata=True, namespace="https://github.com/CoderAgent/SecureAgent")

    # Get the list of retrieved texts
    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n\n\n\n\nMY QUESTION:\n" + query

    # Modify the prompt below as need to improve the response quality
    system_prompt = f"""You are a Senior Software Engineer, specializing in TypeScript.

    Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
    """

    llm_response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )

    return llm_response.choices[0].message.content

In [ ]:
response = perform_rag("What does this github repo do?")

print(response)

This GitHub repo contains a full-stack application that serves as an interface for OpenAI's text generation API. Here's a detailed breakdown of its functionality:

**Core Features:**
1. **AI Text Generation:**
   - Users can input prompts and generate text completions using OpenAI's GPT models
   - Supports adjustable parameters:
     - Temperature (creativity control)
     - Max length (response size)

2. **Model Management:**
   - Fetches available OpenAI models
   - Built-in model configuration system

3. **Context Management:**
   - Central state management using React Context API
   - Tracks generation history
   - Manages model settings across components

**Technical Architecture:**
- **Frontend** (Next.js/React):
  - Context providers for state management
  - Custom hooks for API interactions
  - Interactive UI with:
    - Settings controls
    - Prompt input
    - History display
    - Error handling

- **Backend** (Next.js API routes):
  - Proxy server handling secure OpenAI A